In [7]:
import sys
import config
sys.path.append(config.root)
import glob
import torch
import time
import multiprocessing as mp
import threading as td
import numpy as np
import matplotlib.pyplot as plt
from dataloader.read_preprocess import read_preprocess
from dataloader.parallel_loader import threads_read, threads_scene_dset
from dataloader.loader import scene_tensor_dset



In [8]:
# ----------Data paths-------------- #
paths_as = sorted(glob.glob(config.root+'/data/s1_ascend/*'))
paths_des = sorted(glob.glob(config.root+'/data/s1_descend/*'))
paths_truth = sorted(glob.glob(config.root+'/data/s1_truth/*'))


In [9]:
### -----data read & pre-processing------- ###
scene_list, truth_list = read_preprocess(paths_as=paths_as,\
                                    paths_des=paths_des, paths_truth=paths_truth)
len(scene_list)


20

### Single-threading loading


In [4]:
## ------- single thread ------- 
tra_dset_single = scene_tensor_dset(scene_tensor_list=scene_list, \
                truth_tensor_list=truth_list, transforms=config.transforms_tra, scales=[2048,512,256])
print(tra_dset_single.__len__())
tra_loader_single = torch.utils.data.DataLoader(tra_dset_single, \
                                        batch_size=4, shuffle=True)



20


### Multi-threading loading


In [5]:
threads_tra_dset = threads_scene_dset(scene_list[0:15], \
                                    truth_list[0:15], num_thread=10)

print(threads_tra_dset.__len__())
threads_tra_loader = torch.utils.data.DataLoader(threads_tra_dset, \
                                        batch_size=4, shuffle=True, num_workers=0)


150


### Comparison

In [6]:
for epoch in range(10):
    print(f'epoch {epoch+1}')
    num = 0
    start = time.time()
    # for patch, truth in tra_loader_single:
    for patch, truth in threads_tra_loader:
        num += truth.shape[0]
        # print(num)
    print(f'time/patch:{(time.time()-start)/num},dset size: {num}')

epoch 1
time/patch:0.006246183713277181,dset size: 150
epoch 2
time/patch:0.006181867917378744,dset size: 150
epoch 3
time/patch:0.0064342244466145835,dset size: 150
epoch 4
time/patch:0.005789198875427246,dset size: 150
epoch 5
time/patch:0.005553585688273112,dset size: 150
epoch 6
time/patch:0.006084349950154623,dset size: 150
epoch 7
time/patch:0.006383938789367676,dset size: 150
epoch 8
time/patch:0.0065955845514933265,dset size: 150
epoch 9
time/patch:0.005766787528991699,dset size: 150
epoch 10
time/patch:0.006104575792948405,dset size: 150


In [8]:
start = time.time()
patch_lists, ptruth_lists = threads_read(scene_list, truth_list, num_thread=20)
print(f'time/patch:{(time.time()-start)/len(patch_lists)}')


time/patch:0.004549534320831299


In [8]:
from dataloader.parallel_loader import scenes2patches, threads_read
start = time.time()
patch_list, ptruth_list = scenes2patches(scene_list=scene_list, truth_list =truth_list)
print(f'time/patch_list:{(time.time()-start)/len(ptruth_list)}, dset_size:{len(ptruth_list)}')



time/patch_list:0.004754376411437988, dset_size:20
